In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

In [ ]:
import jax
import jax.numpy as jnp
from flax import nnx
from functools import partial
from jax_tqdm import scan_tqdm

from algorithms.utils import save_state_multiagent

import numpy as np

# jax.config.update("jax_debug_nans", True)

In [ ]:
from ernestogym.envs_jax.multi_agent.env import RECEnv

In [ ]:
def my_env_creator(params, battery_type, env_type='normal'):
    if env_type == 'normal':
        env = RECEnv(params, battery_type)
    return env

In [ ]:
from ernestogym.envs_jax.multi_agent.utils import parameter_generator

In [ ]:
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_cheap.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full_cheap.yaml"
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_half_full_cheap.yaml"

ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"


# world = "ernestogym/envs_jax/multi_agent/world_deg_train.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_demands_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg_passive.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_only_trad_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_all_the_same_debug.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_all_the_same_debug_like_single.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_new_clip.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg_new_clip.yaml"

# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_for_recurrent.yaml"
world = 'ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_for_recurrent_passive_no_gen.yaml'
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_for_recurrent_passive_scaled.yaml"

# battery_type = 'fading'
# battery_type = 'degrading'
battery_type = 'degrading_dropflow'

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    # use_reward_normalization=True

)

params

In [ ]:
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_cheap.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full_cheap.yaml"
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_half_full_cheap.yaml"

ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"



# world = "ernestogym/envs_jax/multi_agent/world_deg_test.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_demands_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg_passive.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_only_trad_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_all_the_same_debug.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_all_the_same_debug_like_single.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_new_clip.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg_new_clip.yaml"

# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_for_recurrent.yaml"
world = 'ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_for_recurrent_passive_no_gen.yaml'
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_for_recurrent_passive_scaled.yaml"

params_testing = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    # use_reward_normalization=True

)

params_testing['step']

In [ ]:
from algorithms.multi_agent_ppo_only_rl import make_train, train_wrapper
from flax.core.frozen_dict import freeze, unfreeze

In [ ]:
num_envs = 4
total_timesteps = 8760 * num_envs * 500

env_type = 'normal'

config = {
    "LR_BATTERIES": 1e-4,
    "LR_BATTERIES_MIN": 1e-6,
    'FRACTION_DYNAMIC_LR_BATTERIES': 0.7,
    "LR_REC": 1e-4,
    "LR_REC_MIN": 1e-6,
    'FRACTION_DYNAMIC_LR_REC': 0.7,
    "NUM_ENVS": num_envs,
    "NUM_STEPS": 8192, #2048 #10000,
    # "TOTAL_TIMESTEPS": 100000 * num_envs,
    "TOTAL_TIMESTEPS": total_timesteps,
    "UPDATE_EPOCHS": 10,
    "NUM_MINIBATCHES": 32, #64,
    "NORMALIZE_ENV": False,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.98,
    "CLIP_EPS": 0.20,
    "ENT_COEF": 0., #0.01,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "NET_ARCH": (64, 32, 16), #(16, 16, 16),
    "LSTM_NET_ARCH": (24, 12),
    "ACTIVATION": 'tanh',
    "LSTM_ACTIVATION": 'tanh',
    "LOGISTIC_FUNCTION_TO_ACTOR": False,
    "ENV_NAME": None,
    "LR_SCHEDULE": 'cosine',
    'DEBUG': False,
    "NORMALIZE_REWARD_FOR_GAE_AND_TARGETS": False,
    "NORMALIZE_TARGETS": False,
    "NORMALIZE_ADVANTAGES": True,
    "ENV_TYPE": env_type,
    'NETWORK_TYPE_BATTERIES': 'actor_critic',
    'NETWORK_TYPE_REC': 'actor_critic',
    'USE_WEIGHT_DECAY': True,
    'NORMALIZE_NN_INPUTS': True
}

rng = jax.random.PRNGKey(42)

env = my_env_creator(params, battery_type, env_type=env_type)
env, train_state = make_train(config, env)

env_testing = my_env_creator(params_testing, battery_type, env_type=env_type)

val_rng = jax.random.PRNGKey(51)
val_num_iters = 8670*5

# out = train_jit(rng)

config = freeze(config)

In [ ]:
config

In [ ]:
env.battery_agents

In [ ]:
env.valorization_incentive_coeff

In [ ]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)

In [ ]:
optimizer_batteries

In [ ]:
networks_batteries

In [ ]:
network_rec

In [ ]:
import time
rng = jax.random.PRNGKey(42)
t0 = time.time()

# with jax.checking_leaks():
out = train_wrapper(env, config, train_state, rng, validate=True, freq_val=10, val_env=env_testing, val_rng=val_rng, val_num_iters=val_num_iters, params=params, path_saving='/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER/trained_agents/')

train_state = out['runner_state'].train_state

print(f"time: {time.time() - t0:.2f} s")

In [ ]:
# jax.tree.map(lambda x: x.shape, out['metrics'])

In [ ]:
# jax.tree.map(lambda x: jnp.isnan(x).any(), out['metrics'])

In [ ]:
# jax.tree.map(lambda x: jnp.isnan(x).any(axis=jnp.arange(1, x.ndim)), out['metrics'])

In [ ]:
# out['metrics']['actions']['REC_agent'][0]

In [ ]:
# out['metrics']['actions']['REC_agent'][1]

In [ ]:
# np.isnan(out['metrics']['actions']['REC_agent'][1]).any()

In [ ]:
# jax.tree.map(lambda x: np.nonzero(np.isnan(x)), out['metrics'])

In [ ]:
# out['metrics']['actions']['REC_agent']

In [ ]:
# out['metrics']['pure_reward']['r_clipping'][3]

In [ ]:
network_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)
network_batteries